In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import codecs

fiqh_file = '/home/jvdzwaan/code/fiqh_corpus/0179MalikIbnAnas.Muwatta.txt'

with codecs.open(fiqh_file, encoding='utf-8') as f:
    text = f.read()

In [ ]:
import re

md_header = re.compile(r'#(.+)')
result = md_header.finditer(text)
print result
for m in result:
    print m.group()

In [ ]:
text = re.sub(r'#META#(.+)\n', '', text)

In [ ]:
text = re.sub(r'######OpenITI#\n', '', text)

In [ ]:
fiqh_file = '/home/jvdzwaan/data/tmp/adh/test/data/0274AhmadBarqi.Mahasin.txt'

with codecs.open(fiqh_file, encoding='utf-8') as f:
    text = f.read()

In [ ]:
quotes = r'@(.+?)@'
pages = r'PageV(\d{2})P(\d+)'
milestones = r'Milestone\d+'

regex = re.compile(pages)
result = regex.finditer(text)
for m in result:
    print m.group()

In [ ]:
clean_file = '/home/jvdzwaan/data/tmp/adh/test/data/0274AhmadBarqi.Mahasin-clean.txt'

with codecs.open(clean_file, encoding='utf-8') as f:
    lines = f.readlines()

In [ ]:
fiqh_file = '/home/jvdzwaan/data/tmp/adh/test/data/0274AhmadBarqi.Mahasin.txt'

with codecs.open(fiqh_file, encoding='utf-8') as f:
    lines_orig = f.readlines()

In [ ]:
print len(lines), len(lines_orig)

In [ ]:
import nltk
from nltk import word_tokenize
from Bio import pairwise2
import six
import re
import string


def tokenize(text):
    tokens = word_tokenize(text)
    
    # nltk tokenizer replaces " (double quotes) with `` and ''.
    # We want to keep the double quotes, so replace them again.
    tokens = ['"' if t == '``' or t == "''" else t for t in tokens]
    
    return tokens
    

def get_spaces_pattern(text):
    # replace regular expressions special characters
    for p in ('(', ')'):
            text = text.replace(p, '#')
    tokens = tokenize(text)
    m = re.match(r'( *)'.join(tokens), text)
    return m.groups()

def merge_sentences(l1, l2):
    if l1 == l2:
        print 'same!'
        return l1
    merged = []

    print 'sentence'
    idx1 = 0
    idx2 = 0
    
    print type(l1)
    print type(l2)
    
    #print len(l1), len(l2)
    # normalize spaces
    l1 = re.sub(r' +', u' ', l1)
    l2 = re.sub(r' +', u' ', l2)
    
    # remove trailing whitespace
    l1 = l1.strip()
    l2 = l2.strip()
    
    #print len(l1), len(l2)
    
    tokens1 = tokenize(l1)
    tokens2 = tokenize(l2)
    
    print len(tokens1), len(tokens2)
    
    print 'l1 is printable: ', all(c in string.printable for c in l1)
    print 'l2 is printable: ', all(c in string.printable for c in l2)
    
    # get spaces patterns
    try:
        text = l1
        spaces1 = get_spaces_pattern(text)
    except:
        print 'Spaces1 error'
        print l1
        print u'#'.join(tokens1)
        print text
        
    try:
        text = l2
        spaces2 = get_spaces_pattern(text)
    except:
        print 'Spaces2 error'
    
    print len(spaces1), len(spaces2)
    print '---'
    
    alignment = pairwise2.align.localms(tokens1,tokens2,2,-1,-0.5,-0.1, gap_char=["GAP"])
    #print alignment
    #print len(alignment)
    for t1, t2 in zip(alignment[0][0], alignment[0][1]):
        print t1, t2
        if t1 == t2:
            #print 'equal'
            #print_char(1, idx1+1, l1[idx1+1])
            merged.append(t1)
            try:
                merged.append(spaces1[idx1])
                if spaces1[idx1] == u' ':
                    print 'adding space based on l1'
            except IndexError:
                # no space to be added
                pass
            idx1 += 1
            idx2 += 1

        elif t1 == 'GAP':
            merged.append(t2)
            try: 
                merged.append(spaces2[idx2])
                if spaces2[idx2] == ' ':
                    print 'adding space based on l2'
            except IndexError:
                pass
            idx2 += 1
            
        elif t2 == 'GAP':
            merged.append(t1)
            
            try:
                merged.append(spaces1[idx1])
                
                if spaces1[idx1] == u' ':
                    print 'adding space based on l1'
            except IndexError:
                pass
            idx1 += 1
        else:
            print 'Problem!'
    merged.append('\n')
    return ''.join(merged)

nltk.download("punkt")

merged = []
for l1, l2 in zip(lines[:10], lines_orig[:10]):
    print l1
    print l2
    merged_sentence = merge_sentences(l1, l2)
    merged.append(merged_sentence)

with codecs.open('/home/jvdzwaan/data/tmp/adh/test/data/0274AhmadBarqi.Mahasin-merged.txt', 'wb', encoding='utf-8') as f:
    f.write(''.join(merged))

In [ ]:
s1 = lines[1]
s2 = lines_orig[1]

s1 = s1.strip()
s2 = s2.strip()

s1 = s1.replace('(', '#')
s1 = s1.replace(')', '#')

tokens1 = word_tokenize(s1)

print s1

In [ ]:
print tokens1

In [ ]:
r'( *)'.join(tokens1)

In [ ]:
m = re.match(r'( *)'.join(tokens1), s1)
print m
print m.groups()

In [ ]:
tokens = ['``', "''", 'bla']
tokens = ['"' if t == '``' or t == "''" else t for t in tokens]
print tokens

In [ ]:
s1 = 'This is a "test" sentence.'
tokens1 = tokenize(s1)
print tokens1
print r'( *)'.join(tokens1)
m = re.match(r'( *)'.join(tokens1), s1)
print m.groups()
print '"{}"'.format(m.group(5))
#for match in m:
#    print m.group()